<a href="https://colab.research.google.com/github/innamast/simulative_python_2/blob/main/%D0%A2%D1%80%D0%B5%D1%82%D0%B8%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-07-22 18:30:21--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-07-22 18:30:21--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv.6’

registrations.csv.6 100%[===================>]  14.57K  --.-KB/s    in 0s      

2025-07-22 18:30:21 (84.7

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [2]:
# Ваше решение
import csv
from datetime import datetime

with open('registrations.csv', 'r') as reg, open('entries.csv') as ent:
  registrations = list(csv.reader(reg, delimiter=';'))
  entries = list(csv.reader(ent, delimiter=';'))
  # Убираем строку заголовков
  registrations.pop(0)
  entries.pop(0)

# Добавляем кол-во дней входа пользователей с момента регистрации
user_days_from_reg = []
for user_ent, date_ent in entries:
  for user_reg, date_reg in registrations:
    if user_ent == user_reg:
      diff = (datetime.strptime(date_ent, '%Y-%m-%d') - \
              datetime.strptime(date_reg, '%Y-%m-%d')).days
      if diff != 0:
        user_days_from_reg.append([user_ent, diff])


In [3]:
jan_users = [user for user in registrations if '2021-01-01' <= user[1] <= '2021-01-31']

# Пользователи, зашедшие на 15-й день
jan_users_15 = []
for user_jan in jan_users:
  for user in user_days_from_reg:
    if user[1] == 15 and user[0] == user_jan[0]:
      jan_users_15.append(user[0])

jan_users_15_unique = set(item for item in jan_users_15)

retention_15_day = round(100.0 * len(jan_users_15_unique)/len(jan_users), 5)
retention_15_day

54.65116

In [4]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [5]:
# Ваше решение
# Пользователи, зашедшие на 30-й день и позже
jan_users_30 = []
for user_jan in jan_users:
  for user in user_days_from_reg:
    if user[1] >= 30 and user[0] == user_jan[0]:
      jan_users_30.append(user[0])

# Отбираем уникальных пользователей
jan_users_30_unique = set(item for item in jan_users_30)

rolling_retention = round(100.0 * len(jan_users_30_unique)/len(jan_users), 5)
rolling_retention

29.06977

In [6]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [7]:
# Ваше решение
def n_day(entries, registrations, n):
  users_n = []
  for user_ent, date_ent in entries:
    for user_reg, date_reg in registrations:
      if user_ent == user_reg and \
        (datetime.strptime(date_ent, '%Y-%m-%d') - \
        datetime.strptime(date_reg, '%Y-%m-%d')).days == n:
        users_n.append([user_ent, date_reg])
  # Отбираем уникальных пользователей
  users_unique = set(item[0] for item in users_n)
  # Отбираем уникальные регистрации
  reg_unique = set(item[0] for item in registrations)

  return round(len(users_unique)/len(reg_unique), 5)

lifetime = 0
for i in range(31):
  lifetime += n_day(entries,registrations, i)
lifetime = round(lifetime, 5)
lifetime

14.804

In [8]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`.  
Если будете считать CR от Retention, ведите расчет от Rolling Retention, таким образом, мы получим всех, кто не заходил в 29 день и после. Ведя расчет от обычного Retention, наоборот - получим только CR в 29 день.


In [9]:
# Ваше решение
users_29 = []
for user in user_days_from_reg:
  if user[1] >= 29:
    users_29.append(user[0])

# Отбираем уникальных пользователей
users_29_unique = set(users_29)
churn_29 = (len(registrations)-len(users_29_unique))/len(registrations)
churn_29

0.509

In [10]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [11]:
# Ваше решение
dec_users = [user for user in entries if '2021-12-01' <= user[1] <= '2021-12-31']
unique_dec_users = set(item[0] for item in dec_users)

dec_mau = len(unique_dec_users)
dec_mau

133

In [12]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [13]:
# Ваше решение
dec_users = [user for user in entries if '2021-12-25' <= user[1] <= '2021-12-31']
unique_dec_users = set(item[0] for item in dec_users)

dec_wau = len(unique_dec_users)
dec_wau

84

In [14]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [15]:
# Ваше решение
dec_users = [user for user in entries if user[1] == '2021-12-31']
unique_dec_users = set(item[0] for item in dec_users)

dec_dau = len(unique_dec_users)
dec_dau

47

In [16]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [17]:
d = {'key1':1,'key2':14,'key3':47}
sum(d.values())

62

In [18]:
# Ваше решение
from collections import defaultdict
from datetime import datetime
users = []
counts = defaultdict(set)

for id, date in entries:
  date_YM = date[:7]
  users.append([id, date_YM])
  # Ключ в формате "ГОД-НОМЕР_НЕДЕЛИ"
  key = f"{date_YM}"
  counts[key].add(id)

count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)

sum_total = 0
for key, value in count_of_unique_user_ids.items():
    sum_total += value

avg_mau = round(sum_total/len(count_of_unique_user_ids), 5)
avg_mau

102.58333

In [19]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [20]:
# Ваше решение
from collections import defaultdict
from datetime import datetime
users = []
counts = defaultdict(set)

for id, date in entries:
  date_obj = datetime.strptime(date, '%Y-%m-%d')
# Номер недели в году и год
  year, week_num, _ = date_obj.isocalendar()
  users.append([id, week_num])
  # Ключ в формате "ГОД-НОМЕР_НЕДЕЛИ"
  key = f"{year}-{week_num}"
  counts[key].add(id)

count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)

sum_total = 0
for key, value in count_of_unique_user_ids.items():
    sum_total += value

avg_wau = round(sum_total/len(count_of_unique_user_ids), 5)
avg_wau

89.86792

In [21]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [22]:
# Ваше решение
from collections import defaultdict
from datetime import datetime

unique_users = set(tuple(item) for item in entries)
# Словарь для подсчёта
date_count_dict = defaultdict(int)

# Считаем количество элементов для каждой даты
for item in unique_users:
    date = item[1]
    date_count_dict[date] += 1

date_count_dict = dict(date_count_dict)
total_sum_am = sum(date_count_dict.values())
avg_dau = round(total_sum_am/365, 5)
avg_dau

40.5589

In [23]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
